# Transformation des données des tables raw_data vers la table chantier pour le schéma public

In [ ]:
# import des librairies nécessaires pour l'exploration
import pandas as pd
from pandas_profiling import ProfileReport
from dotenv import load_dotenv

from ditp.postgresql.connector import PostgreSQLConnector

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 350)

In [ ]:
# Chargement des variables d'environnement
load_dotenv('../../.env')

## Lire des données d'une table

In [ ]:
def read_table(table: str, schema = 'public') -> pd.DataFrame:
    pg_connector = PostgreSQLConnector()
    table = pd.read_sql_query(
        sql=f'''
                SELECT *
                FROM {schema}.{table}
                ''',
        con=pg_connector.connection
    )
    return table

In [ ]:
def execute_sql(sql_request: str) -> pd.DataFrame:
    pg_connector = PostgreSQLConnector()
    table = pd.read_sql_query(
        sql=sql_request,
        con=pg_connector.connection
    )
    return table

### Lecture des tables 

In [ ]:
metadata_chantier = read_table('metadata_chantier', 'raw_data')
fact_progress_chantier = read_table('fact_progress_chantier', 'raw_data')
dim_structures = read_table('dim_structures', 'raw_data')
dim_tree_nodes = read_table('dim_tree_nodes', 'raw_data')

In [ ]:
prof = ProfileReport(metadata_chantier)
prof.to_file(output_file='rapport_metadata_chantier.html')

In [ ]:
prof = ProfileReport(fact_progress_chantier)
prof.to_file(output_file='rapport_fact_progress_chantier.html')

In [ ]:
prof = ProfileReport(dim_structures)
prof.to_file(output_file='rapport_dim_structures.html')

In [ ]:
prof = ProfileReport(dim_tree_nodes)
prof.to_file(output_file='rapport_dim_tree_nodes.html')

### Requête SQL

#### Dfakto chantier

Récupération des données des chantiers avec les données issues de dfakto

In [ ]:
fact_progress_chantier.shape

In [ ]:
sql_request_dfakto = f''' 
    SELECT fpc.tree_node_id, 
        fpc.bounded_progress, 
        fpc.progress, 
        dtn.structure_id, 
        dtn.maturity_id, 
        dtn.tree_node_code,
        split_part(dtn.tree_node_code, '-', 1) as code_chantier,
        split_part(dtn.tree_node_code, '-', 2) as code_region, 
        dtn.tree_node_name, 
        dtn.tree_node_status, 
        ds.top_level_id, 
        ds.structure_name, 
        ds.structure_level 
    FROM raw_data.fact_progress_chantier fpc 
        JOIN raw_data.dim_tree_nodes dtn ON fpc.tree_node_id = dtn.tree_node_id 
        JOIN raw_data.dim_structures ds ON dtn.structure_id = ds.structure_id
        WHERE  ds.structure_name IN ('Réforme', 'Région')
'''

In [ ]:
dfakto_chantiers = execute_sql(sql_request_dfakto)
#dfakto_chantiers = dfakto_chantiers.T.drop_duplicates().T
print(dfakto_chantiers.shape)
print(dfakto_chantiers[['tree_node_code']].value_counts(ascending=True).reset_index(name='count').shape)

In [ ]:
dfakto_chantiers.structure_name.value_counts()

In [ ]:
dfakto_chantiers.head(200)

#### Public chantier

Dans cette partie, on souhaite construire la table `public.chantier` pour les niveaux : réforme (national), régions et départements

##### Table public.chantier au niveau National 

In [ ]:
sql_chantier_national = f'''
    WITH dfakto_chantier AS (
        SELECT fact_progress_chantier.tree_node_id,
            fact_progress_chantier.bounded_progress,
            split_part(dim_tree_nodes.tree_node_code, '-', 1) as code_region,
            split_part(dim_tree_nodes.tree_node_code, '-', 2) as code_chantier
        FROM raw_data.fact_progress_chantier 
            JOIN raw_data.dim_tree_nodes ON fact_progress_chantier.tree_node_id = dim_tree_nodes.tree_node_id 
            JOIN raw_data.dim_structures ON dim_tree_nodes.structure_id = dim_structures.structure_id 
        WHERE dim_structures.structure_name='Réforme'
    )
    SELECT m_chantier.chantier_id as id,
        m_chantier.ch_nom as nom,
        m_chantier.ch_per as perimetre_ids,
        m_chantier."porteur_ids_noDAC" as porteurs_ids,
        m_chantier."porteur_ids_DAC" as porteurs_dac_ids,
        d_chantier.bounded_progress as taux_avancement,
        m_zone.nom as zone_nom,
        m_zone.zone_code as code_insee
    FROM raw_data.metadata_chantier m_chantier
        LEFT JOIN dfakto_chantier d_chantier ON m_chantier.ch_perseverant = d_chantier.code_chantier
        JOIN raw_data.metadata_zone m_zone ON m_zone.zone_id = 'FRANCE';
'''

In [ ]:
chantier_national = execute_sql(sql_chantier_national)
chantier_national.shape

In [ ]:
chantier_national.describe(include='all')

In [ ]:
chantier_national.head(200)

C'est le chantier UQP qui est perséverant mais n'a pas de réforme au niveau national

##### Table public.chantier aux niveaux des Régions et Départements

In [ ]:
sql_chantier_region_departement = f'''
    WITH dfakto_chantier AS (
        SELECT fpc.tree_node_id,
            fpc.bounded_progress,
            split_part(dtn.tree_node_code, '-', 1) as code_chantier,
            split_part(dtn.tree_node_code, '-', 2) as code_region
        FROM raw_data.fact_progress_chantier fpc 
            JOIN raw_data.dim_tree_nodes dtn ON fpc.tree_node_id = dtn.tree_node_id 
            JOIN raw_data.dim_structures ds ON dtn.structure_id = ds.structure_id 
        WHERE ds.structure_name IN ('Région', 'Département')
    )
    SELECT m_chantier.chantier_id as id,
        m_chantier.ch_nom as nom,
        m_chantier.ch_per as ids_perimetre,
        m_chantier."porteur_ids_noDAC" as porteurs_ids,
        m_chantier."porteur_ids_DAC" as porteurs_dac_ids,
        d_chantier.bounded_progress as taux_avancement,
        m_zone.nom as zone_nom,
        m_zone.zone_code as code_insee
    FROM raw_data.metadata_chantier m_chantier
        LEFT JOIN dfakto_chantier d_chantier ON m_chantier.ch_perseverant = d_chantier.code_chantier
        JOIN raw_data.metadata_zone m_zone ON m_zone.zone_id = d_chantier.code_region;

'''

In [ ]:
chantier_reg_dept = execute_sql(sql_chantier_region_departement)
chantier_reg_dept.shape

In [ ]:
chantier_reg_dept.describe(include='all')

In [ ]:
chantier_reg_dept.head(200)